In [2]:
import pandas as pd
import json
import re
import pymongo
import jieba
import operator

In [4]:
IP:''
user:''
password:''

In [3]:
#連接 mongodb

client = pymongo.MongoClient(host='', port=27017)

In [4]:
#mongodb帳號密碼

client.admin.authenticate('root', '')



#設定名稱為test的資料庫 也可用 db = client['test']

db = client.raw_article




In [5]:
client.database_names()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['ETL', 'admin', 'config', 'local', 'raw_article', 'test']

In [6]:
#顯示db中所有collections
db.list_collection_names()

['article_social', 'article_web', 'article_news']

In [6]:
#drop collection
#db.article_socialjson.drop()

In [7]:
article_social=db.article_social.find()
article_web=db.article_web.find()
article_news=db.article_news.find()

In [9]:
df_article_social=pd.DataFrame(list(article_social))
df_article_web=pd.DataFrame(list(article_web))
df_article_news=pd.DataFrame(list(article_news))

In [11]:
df_article_social[:1]

,_id,content
0,5dd39ed135cc75f4e568cc8f,#公告 信用卡板板主\n本板板主為：信用卡板小天使https://www.dcard.tw/...


In [10]:
#斷詞
#df_article_web
n=len(df_article_web)
alltext=[]
stopset=set()
stop2=['\/n','','\\n','\n']


with open(r'E:/專題/credit_card/DB104_hau/stop.txt','r',encoding='utf-8') as s:
    for line in s:
        stopset.add(line.strip('\n'))
jieba.load_userdict(r'E:/專題/credit_card/DB104_hau/userdict.txt')
for i in range(n):
    all_seg={'seg':''}
    segn=[]
    wf={}
    text= str(df_article_web.loc[i]['content'])
    cut=jieba.lcut(text,cut_all=False)
    #print(cut)
    for j in cut:
            if j not in (stopset and stop2):
             segn.append(j)
    all_seg['seg']=segn
    alltext.append(all_seg)

#     for w in seg.split(' '):
#         if w not in wf:
#             wf[w]=1
#         else:
#             wf[w]+=1
print(alltext[0:5])
    #word_list=[wf.items() for k in wf if k not in stop2 ]
#     word_list=sorted(wf.items(),key=lambda a :a[1],reverse=True)
#     df_article_web['word_freq'][i]=word_list

#del df_article_web['_id']
#del df_article_web['content']
#df_article_web[:2]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.928 seconds.
Prefix dict has been built succesfully.


[{'seg': ['玉山', 'Pi', '拍錢包', '信用卡', '》', '國內', '4.5%', '.', '海外', '3%', '.', '繳稅', '1%', '持續', '坐', '穩神', '卡寶座', '！', 'May16Thu201923', ':', '12', '玉山', 'Pi', '拍錢包', '信用卡', '》', '國內', '4.5%', '.', '海外', '3%', '.', '繳稅', '1%', '持續', '坐', '穩神', '卡寶座', '！', '?', '玉山', 'Pi', '拍錢包', '信用卡', '從', '去年', '三季', '推出', '，', '除了', '基本', '回饋', '最高', '4.5%', '、', '繳水電', '也', '有', '回饋', '的', '強大', '優勢', '，', '且', '卡片', '限定', '短期', '優惠', '也', '能', '有效', '提升', '動卡率', '，', '讓', '人', '感覺', '銀行', '十足', '誠', '意在', '經營', '這張', '卡片', '，', '也', '因此', '，', '這', '一年', '來', '討論度', '不降反升', '熱度', '依舊', '！', '不管', '到', '哪個', '信用卡', '的', '論壇', '搜尋關鍵', '字', '，', '每天', '都', '有人', '在', '分享', 'Pi', '拍錢包', '信用卡', '，', '分享', '數多到', '要', '花', '好', '幾個', '小時', '消化', '完', '！', '?', '這張', '卡', '也', '成為', '我', '跟', '讀者', '私訊', '討論', '次數', '最', '多', '的', '卡片', '前', '五名', '，', '有', '網友', '曾經', '跟', '我', '討論', '到', '半夜', '兩點', '，', '這個', '經驗', '真的', 'Neveragain', '，', '由', '於', 'Pi', '拍錢包', '信用卡', '也', '是', '我', '今年', '上半年', '的', 

In [11]:
with open('j_article_web.json', 'w', encoding='utf-8') as w:
     json.dump(alltext,w,ensure_ascii=False)

In [21]:
db_ETL = client.ETL
#collsoc = db.article_social
collweb = db_ETL.article_web_seg
collnews =db_ETL.article_news_seg
collsocial = db_ETL.article_social_seg


In [24]:
#將dataframe轉成能存入mongodb的json格式

#data_social = json.loads(df_article_social.T.to_json()).values()
#data_news = json.loads(df_article_news.T.to_json()).values()
#data_web = json.loads(df_article_web.T.to_json()).values()

#data_social[:2]

In [22]:
#將資料存入mongodb
collsocial.insert_many(alltext_soc)



In [23]:
collnews.insert_many(alltext_news)

In [24]:
#collweb.insert_many(data_web)
collweb.insert_many(alltext)

In [25]:
db_ETL.list_collection_names()

['article_tag_soc',
 'article_news_seg',
 'article_tag_news',
 'article_web_seg',
 'article_social_seg',
 'article_tag_web']

In [35]:
list(db_ETL.article_web_seg.find())[:10]

[{'_id': ObjectId('5dd685cf7b56f49072d672b4'),
  'seg': ['玉山',
   'Pi',
   '拍錢包',
   '信用卡',
   '》',
   '國內',
   '4.5%',
   '.',
   '海外',
   '3%',
   '.',
   '繳稅',
   '1%',
   '持續',
   '坐',
   '穩神',
   '卡寶座',
   '！',
   'May16Thu201923',
   ':',
   '12',
   '玉山',
   'Pi',
   '拍錢包',
   '信用卡',
   '》',
   '國內',
   '4.5%',
   '.',
   '海外',
   '3%',
   '.',
   '繳稅',
   '1%',
   '持續',
   '坐',
   '穩神',
   '卡寶座',
   '！',
   '?',
   '玉山',
   'Pi',
   '拍錢包',
   '信用卡',
   '從',
   '去年',
   '三季',
   '推出',
   '，',
   '除了',
   '基本',
   '回饋',
   '最高',
   '4.5%',
   '、',
   '繳水電',
   '也',
   '有',
   '回饋',
   '的',
   '強大',
   '優勢',
   '，',
   '且',
   '卡片',
   '限定',
   '短期',
   '優惠',
   '也',
   '能',
   '有效',
   '提升',
   '動卡率',
   '，',
   '讓',
   '人',
   '感覺',
   '銀行',
   '十足',
   '誠',
   '意在',
   '經營',
   '這張',
   '卡片',
   '，',
   '也',
   '因此',
   '，',
   '這',
   '一年',
   '來',
   '討論度',
   '不降反升',
   '熱度',
   '依舊',
   '！',
   '不管',
   '到',
   '哪個',
   '信用卡',
   '的',
   '論壇',
   '搜尋關鍵',
   '字',
   '，',
   

In [ ]:
client.database_names()

In [19]:
db_ETL=client.ETL

In [20]:
#db_ETL.article_news_seg.drop()
#db_ETL.article_web_seg.drop()
#db_ETL.article_social_seg.drop()
#drop collection
#db.article_socialjson.drop()

In [ ]:
db_ETL